<a href="https://colab.research.google.com/github/MilciadesGonzalez/TradeLaVirginia/blob/main/Feg_Semanal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
crudo = pd.read_excel('/content/drive/Shareddrives/Trade Mktg/Trade/3 - FEG Venta/2 - Reporte Quincenal/Feg Semanal con Python/1 - Crudo/EncuestasFEG.xlsx')
vendedores = pd.read_excel('/content/drive/Shareddrives/Trade Mktg/Trade/3 - FEG Venta/2 - Reporte Quincenal/Feg Semanal con Python/2 - DatosExternos/DatosExternos.xlsx', sheet_name='Vendedores')
usuariosSac = pd.read_excel('/content/drive/Shareddrives/Trade Mktg/Trade/3 - FEG Venta/2 - Reporte Quincenal/Feg Semanal con Python/2 - DatosExternos/DatosExternos.xlsx', sheet_name='UsuariosSAC')
recorrido = pd.read_excel('/content/drive/Shareddrives/Trade Mktg/Trade/3 - FEG Venta/2 - Reporte Quincenal/Feg Semanal con Python/2 - DatosExternos/DatosExternos.xlsx', sheet_name='Recorrido')
kgCartera = pd.read_excel('/content/drive/Shareddrives/Trade Mktg/Trade/3 - FEG Venta/2 - Reporte Quincenal/Feg Semanal con Python/2 - DatosExternos/DatosExternos.xlsx', sheet_name='KgCartera')
kgConsolidador = pd.read_excel('/content/drive/Shareddrives/Trade Mktg/Trade/3 - FEG Venta/2 - Reporte Quincenal/Feg Semanal con Python/2 - DatosExternos/DatosExternos.xlsx', sheet_name='KgConsolidador')
kgBoca = pd.read_excel('/content/drive/Shareddrives/Trade Mktg/Trade/3 - FEG Venta/2 - Reporte Quincenal/Feg Semanal con Python/2 - DatosExternos/DatosExternos.xlsx', sheet_name='KgBoca')
regionales = pd.read_excel('/content/drive/Shareddrives/Trade Mktg/Trade/3 - FEG Venta/2 - Reporte Quincenal/Feg Semanal con Python/2 - DatosExternos/DatosExternos.xlsx', sheet_name='Regionales')
pondera = pd.read_excel('/content/drive/Shareddrives/Trade Mktg/Trade/3 - FEG Venta/2 - Reporte Quincenal/Feg Semanal con Python/2 - DatosExternos/DatosExternos.xlsx', sheet_name='Pondera')
resumenVend = pd.read_excel('/content/drive/Shareddrives/Trade Mktg/Trade/3 - FEG Venta/2 - Reporte Quincenal/Feg Semanal con Python/2 - DatosExternos/DatosExternos.xlsx', sheet_name='ResumenVendedor')

In [ ]:
base = pd.merge(crudo, vendedores, on='RealizadaPor', how='left')
recorridoV = pd.merge(recorrido, kgConsolidador, on='Codigo', how='left')
recorridoV = pd.merge(recorridoV, kgBoca, on='Codigo', how='left')
recorridoV['Kg'] = np.where((recorridoV.KgC > 0),recorridoV.KgC,recorridoV.KgB)
recorridoV = recorridoV.drop(['KgC','KgB'], axis=1)

In [ ]:
controlFotos = base[['Sucursal','Supervisor','Codigo','Cliente','NombreVendedor','CodVendedor','TipoEncuesta','TituloPregunta','Valor','Url','FechaRelvamiento']]
controlFotos = pd.merge(controlFotos, regionales, on='Sucursal', how='left')

In [ ]:
auxPuntos = pd.merge(controlFotos, pondera, on='TituloPregunta', how='left')
puntos = auxPuntos[['Sucursal','Supervisor','Codigo','Cliente','NombreVendedor','CodVendedor','Valor','Pondera1','Pondera2','TipoEncuesta']]
puntos['Valor'] = puntos['Valor']/100
puntos['PjeFinal'] = puntos['Valor']*puntos['Pondera2']
puntos['PjeCategoría'] = puntos['Valor']*puntos['Pondera1']

<ipython-input-5-6155b26c1acc>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  puntos['Valor'] = puntos['Valor']/100
<ipython-input-5-6155b26c1acc>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  puntos['PjeFinal'] = puntos['Valor']*puntos['Pondera2']
<ipython-input-5-6155b26c1acc>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs

In [ ]:
puntosCateg = puntos.pivot_table(index='Codigo', columns='TipoEncuesta', values='PjeCategoría', aggfunc=np.sum)
puntosCateg = puntosCateg.reset_index()

In [ ]:
TD_puntos = puntos.pivot_table(index=['Codigo','CodVendedor'], values='PjeFinal', aggfunc=np.sum)
TD_puntos = TD_puntos.reset_index()

In [ ]:
TD_puntos = pd.merge(TD_puntos, recorridoV, on='Codigo', how='left')
TD_puntos['Kg_70%'] = np.where((TD_puntos['PjeFinal'] >= 0.695),'70%o+','-de70%')
nota70 = TD_puntos.pivot_table(index=['CodVendedor_x','Kg_70%'], values='Kg', aggfunc=np.sum)
nota70 = nota70.reset_index()
kilos70 = nota70[nota70['Kg_70%']=='70%o+']
kilos70 = kilos70.rename(columns={'CodVendedor_x':'CodVendedor', 'Kg':'KG_70%'})

In [ ]:
control = TD_puntos[['Sucursal','Supervisor','Codigo','NombreVendedor','CodVendedor_x','PjeFinal']]
control = pd.merge(control, puntosCateg, on='Codigo', how='left')
auxBase = base[['Codigo','Cliente',]]
control = pd.merge(control, auxBase, on='Codigo', how='left')
control = pd.merge(control, regionales, on='Sucursal', how='left')
control = control.drop_duplicates()
control['EncHechas'] = control[['FEGCacao', 'FEGCafe', 'FEGCondimentos', 'FEGEdulcorantes','FEGPSecas', 'FEGTe']].count(axis=1)

<ipython-input-9-4f03e65312c8>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  control['EncHechas'] = control[['FEGCacao', 'FEGCafe', 'FEGCondimentos', 'FEGEdulcorantes','FEGPSecas', 'FEGTe']].count(axis=1)


In [ ]:
control = control.rename(columns={'CodVendedor_x':'CodVendedor','PjeFinal':'NotaTotal'})
control = control[['Sucursal','Supervisor','Codigo','Cliente','NombreVendedor','CodVendedor','FEGCacao','FEGCafe','FEGCondimentos','FEGEdulcorantes','FEGPSecas','FEGTe','EncHechas','NotaTotal','Regional']]
control.fillna('Hacer', inplace=True)

In [ ]:
TD_encuestasHechas = base.pivot_table(index=['Sucursal','Supervisor','Codigo','Cliente','NombreVendedor','CodVendedor','RealizadaPor'], columns=['TipoEncuesta'], values='Valor', aggfunc=np.mean)
TD_encuestasHechas =  TD_encuestasHechas.reset_index()
TD_encuestasHechas['FEGCacao'] = np.where((TD_encuestasHechas.FEGCacao > 0),1,np.nan)
TD_encuestasHechas['FEGCafe'] = np.where((TD_encuestasHechas.FEGCafe > 0),1,np.nan)
TD_encuestasHechas['FEGCondimentos'] = np.where((TD_encuestasHechas.FEGCondimentos > 0),1,np.nan)
TD_encuestasHechas['FEGEdulcorantes'] = np.where((TD_encuestasHechas.FEGEdulcorantes > 0),1,np.nan)
TD_encuestasHechas['FEGPSecas'] = np.where((TD_encuestasHechas.FEGPSecas > 0),1,np.nan)
TD_encuestasHechas['FEGTe'] = np.where((TD_encuestasHechas.FEGTe > 0),1,np.nan)

In [ ]:
TD_encuestasHechas['Hechas'] = TD_encuestasHechas[['FEGCacao','FEGCafe','FEGCondimentos','FEGEdulcorantes','FEGPSecas','FEGTe']].sum(axis=1, skipna=True)
TD_encuestasHechas['Total'] = 6
TD_encuestasHechas['FEGCacao'] = np.where((TD_encuestasHechas.FEGCacao > 0),'Hecha',np.nan)
TD_encuestasHechas['FEGCafe'] = np.where((TD_encuestasHechas.FEGCafe > 0),'Hecha',np.nan)
TD_encuestasHechas['FEGCondimentos'] = np.where((TD_encuestasHechas.FEGCondimentos > 0),'Hecha',np.nan)
TD_encuestasHechas['FEGEdulcorantes'] = np.where((TD_encuestasHechas.FEGEdulcorantes > 0),'Hecha',np.nan)
TD_encuestasHechas['FEGPSecas'] = np.where((TD_encuestasHechas.FEGPSecas > 0),'Hecha',np.nan)
TD_encuestasHechas['FEGTe'] = np.where((TD_encuestasHechas.FEGTe > 0),'Hecha',np.nan)
TD_encuestasHechas['%'] = TD_encuestasHechas['Hechas']/TD_encuestasHechas['Total']

In [ ]:
sumaHechas = TD_encuestasHechas.pivot_table(index='CodVendedor', values='Hechas', aggfunc=np.sum)
bocasVisitadas = TD_encuestasHechas.pivot_table(index='CodVendedor', values='Codigo', aggfunc='count')
bocasObj = recorridoV.pivot_table(index='CodVendedor', values='Codigo', aggfunc='count')
kgObj = recorridoV.pivot_table(index='CodVendedor', values='Kg', aggfunc=np.sum)

In [ ]:
sumaHechas =  sumaHechas.reset_index()
bocasVisitadas =  bocasVisitadas.reset_index()
bocasObj =  bocasObj.reset_index()
kgObj =  kgObj.reset_index()
sumaHechas = sumaHechas.rename(columns={'Hechas':'EncuestasHechas'})
bocasVisitadas = bocasVisitadas.rename(columns={'Codigo':'BocasVisitadas'})
bocasObj = bocasObj.rename(columns={'Codigo':'ObjBocas'})
bocasObj['ObjEnc'] = 6 * bocasObj['ObjBocas']

In [ ]:
resumen = pd.merge(resumenVend, sumaHechas, on='CodVendedor', how='left')
resumen = pd.merge(resumen, bocasVisitadas, on='CodVendedor', how='left')
resumen = pd.merge(resumen, bocasObj, on='CodVendedor', how='left')
resumen = pd.merge(resumen, kgObj, on='CodVendedor', how='left')
resumen['%_EncHechas'] = resumen['EncuestasHechas']/resumen['ObjEnc']
resumen['%_BocasHechas'] = resumen['BocasVisitadas']/resumen['ObjBocas']
resumen = pd.merge(resumen, usuariosSac, on='CodVendedor', how='left')
resumen = pd.merge(resumen, kgCartera, on='VendedorSAC', how='left')
resumen = resumen.drop('VendedorSAC', axis=1)
resumen['RatioFeg'] = resumen['Kg']/resumen['KgCartera']
resumen = pd.merge(resumen, kilos70, on='CodVendedor', how='left')
resumen['%Kg_arriba_70'] = resumen['KG_70%']/resumen['Kg']

In [ ]:
control['Codigo'] = '0' + control['Codigo'].astype(str)
control.to_excel('TD_Control_.xlsx', index=False)
!cp TD_Control_.xlsx "/content/drive/Shareddrives/Trade Mktg/Trade/3 - FEG Venta/2 - Reporte Quincenal/Feg Semanal con Python"

In [ ]:
controlFotos['Codigo'] = '0' + controlFotos['Codigo'].astype(str)
controlFotos.to_excel('controlFotos_.xlsx', index=False)
!cp controlFotos_.xlsx "/content/drive/Shareddrives/Trade Mktg/Trade/3 - FEG Venta/2 - Reporte Quincenal/Feg Semanal con Python"

In [ ]:
TD_Resumen = resumen.pivot_table(index=['Sucursal','Supervisor','NombreVendedor'], values=['%_EncHechas','%_BocasHechas','%Kg_arriba_70','KgCartera','Kg','RatioFeg','ObjBocas'], aggfunc=np.mean)
TD_Resumen = TD_Resumen.reset_index()
TD_Resumen = TD_Resumen[['Sucursal','Supervisor','NombreVendedor','%_EncHechas','%_BocasHechas','%Kg_arriba_70','KgCartera','Kg','RatioFeg','ObjBocas']]

In [ ]:
TD_Resumen.to_excel('TD_Resumen_.xlsx', index=False)
!cp TD_Resumen_.xlsx "/content/drive/Shareddrives/Trade Mktg/Trade/3 - FEG Venta/2 - Reporte Quincenal/Feg Semanal con Python"